<a href="https://colab.research.google.com/github/OdysseusPolymetis/digital_classics_course/blob/main/7_word_vectors_lat_gk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Vecteurs de mots statiques pour le grec ancien et le latin**
---


Dans ce notebook, vous pourrez prendre le texte de votre choix (grec ou latin, mais limité) ici : [Perseus Treebank](https://github.com/PerseusDL/treebank_data.git).

Nous allons utiliser cette banque parce que la plupart des données a été à peu près contrôlée, et correctement lemmatisée. Ça nous évitera le traitement par des modules tiers.

## 1. Préparation des phrases


In [ ]:
!git clone https://github.com/PerseusDL/treebank_data.git

J'ajoute une liste de mots outils : décommentez la commande dont vous avez besoin en fonction de votre choix de langue.

In [ ]:
!wget https://raw.githubusercontent.com/OdysseusPolymetis/digital_classics_course/main/stopwords_gk.txt
#!wget https://raw.githubusercontent.com/OdysseusPolymetis/digital_classics_course/main/stopwords_lat.txt

In [ ]:
from pathlib import Path
import xml.etree.ElementTree as ET
import os
import glob

Deux fonctions à suivre, une qui charge les mots outils (attention au chemin de fichier quand vous l'appellerez), et l'autre qui parcourt les dossiers (même remarque).

In [ ]:
def load_stopwords(path_txt):
    with open(path_txt, "r", encoding="utf-8") as f:
        return {line.strip() for line in f if line.strip() and not line.lstrip().startswith("#")}

Dans cette fonction, vous avez l'option `stopwords`, que vous pourrez activer ou non à l'appel (par défaut ce sera activé).

In [ ]:
def lemmas_by_sentence(xml_path, include_punct=False, stopwords=None):
    xml_path = Path(xml_path)
    sentences = []
    current = []

    for event, elem in ET.iterparse(xml_path, events=("start", "end")):
        tag = elem.tag

        if event == "start" and tag == "sentence":
            current = []

        elif event == "end" and tag == "word":
            postag = elem.attrib.get("postag", "")
            if not include_punct and postag.startswith("u"):
                pass
            else:
                lemma = elem.attrib.get("lemma")
                if lemma:
                    if not stopwords or lemma not in stopwords:
                        current.append(lemma)

        elif event == "end" and tag == "sentence":
            sentences.append(current)
            elem.clear()

    return sentences

Pour la première variable `text_folder`, vous devez mettre le chemin exact qui mène au dossier auteur (si vous voulez traiter un auteur en entier) ou au dossier texte. La seconde variable `auteur` devra contenir le code auteur de votre choix (code urn sur phi ou tlg, exemple : pour Homère, tlg0012).

In [ ]:
text_folder = "/content/treebank_data/v2.1/Greek/texts"
auteur = "tlg0012"

Ici, selon que vous utilisez le grec ou le latin, décommentez la ligne dont vous avez besoin, commentez l'autre.

In [ ]:
stopwords = load_stopwords("/content/stopwords_gk.txt")
#stopwords = load_stopwords("/content/stopwords_lat.txt")

In [ ]:
files = sorted(glob.glob(os.path.join(text_folder, "**", f"{auteur}.*.xml"), recursive=True))

In [ ]:
sentences = [sentence for f in files for sentence in lemmas_by_sentence(f, stopwords=stopwords)]

On vérifie simplement que les phrases sont bien prises en charge :

In [ ]:
len(sentences)

15138

## 2. Vectorisation

Pour effectuer la vectorisation, nous allons passer par une librairie classique, gensim (dispo [ici](https://radimrehurek.com/gensim/)).

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec

La cellule qui suit va permettre de constituer le modèle. Le temps que ça prendra dépendra des paramètres que vous mettrez. Voilà une brève explication.


*   `sentences` : c'est la liste de phrases lemmatisées
*   `min_count` : c'est le nombre minimal de fois qu'un mot doit apparaître pour être pris en compte
*   `max_vocab_size` : c'est le nombre de mots max qui va être compris dans le modèle
*   `negative` : le modèle voit des paires vraies (mot en contexte) et, pour chaque paire vraie, il voit aussi des paires fausses (ce qu'on appelle adversarial training) choisies au hasard, ici X faux voisins par vrai voisin : théoriquement, plus cette valeur est haute, plus l’apprentissage est précis, et coûteux
*   `epochs` : nombre de fois où le modèle parcourt tout le corpus







In [ ]:
model = Word2Vec(sentences, min_count=2, max_vocab_size=10000, negative=50, epochs=300)

In [ ]:
model.wv.most_similar(positive=["γυνή","θεός"], negative=["ἀνήρ"],topn=10)

In [ ]:
model.wv.most_similar('θεός',topn=20)

Et voici un petit bout de code pour exporter vos données et les visualiser plus aisément. Vous pouvez vous rendre sur le site de projection de [tensorflow](https://projector.tensorflow.org/), appuyer sur le bouton `load` sur la gauche, mettre le fichier de `1_vecteurs.tsv` en première option du pop-up, et le fichier `2_metadonnees.tsv` dans la seconde.

In [ ]:
with open("/content/1_vecteurs.tsv", 'w') as file_vectors, open("/content/2_metadonnees.tsv", 'w') as file_metadata:
    for word in model.wv.index_to_key:
        file_vectors.write('\t'.join([str(x) for x in model.wv[word]]) + "\n")
        file_metadata.write(word + "\n")

# **Avec votre propre texte**

Ici même principe, mais avec un txt. Plus le txt est long, mieux ça marchera.

In [ ]:
!pip install stanza

Dans la cellule suivante, veillez à changer `"/content/stopwords_lat.txt"` en `"/content/stopwords_grc.txt"` en fonction de la langue que vous aurez choisie

In [ ]:
stopwords = open("/content/stopwords_lat.txt",'r',encoding="utf8").read().split("\n")

Dans la cellule suivante, vous allez pouvoir choisir un fichier à traiter.

In [ ]:
from google.colab import files
uploaded = files.upload()

Ici vous devez mettre le code langue pour l'analyse.

In [ ]:
LANG = "la"

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device utilisé :", DEVICE, "| Langue :", LANG)

In [ ]:
stanza.download(LANG)
nlp = stanza.Pipeline(
    lang=LANG,
    processors="tokenize,lemma",
    use_gpu=(DEVICE == "cuda"),
    device=DEVICE
)

Dans cette cellule, vous veillerez à mettre le bon nom de fichier (votre fichier). Attention, pas d'espaces ou de caractères spéciaux dans le titre.

In [ ]:
filepath_of_text = "/content/votreTexte.txt"

In [ ]:
full_text = open(filepath_of_text, encoding="utf-8").read()

In [ ]:
def batch_process_to_lemmas(text, nlp, batch_size=100):
    paragraphs = text.split('\n')
    batches = [paragraphs[i:i + batch_size] for i in range(0, len(paragraphs), batch_size)]

    sentences_lemmas = []

    for batch in batches:
        batch_text = '\n'.join(batch)
        doc = nlp(batch_text)
        for sentence in doc.sentences:
            sentence_lemmas = []
            for word in sentence.words:
                if word.lemma is not None and word.lemma not in stopwords:
                    sentence_lemmas.append(word.lemma.lower())
            sentences_lemmas.append(sentence_lemmas)

    return sentences_lemmas

In [ ]:
sentences = batch_process_to_lemmas(full_text, nlp_stanza)

Cette cellule-là vous permet d'entraîner le modèle. Pensez à regarder les différents paramètres.

In [ ]:
model = Word2Vec(sentences, min_count=2, max_vocab_size=10000, negative=10, epochs=300)

Ici petite cellule de test pour voir si tout fonctionne.

In [ ]:
model.wv.most_similar('amicitia',topn=50)

Ici c'est l'export de vos fichiers.

In [ ]:
export_dir = "/content/output"
os.makedirs(export_dir, exist_ok=True)

In [ ]:
with open("/content/output/vecteurs.tsv", 'w') as file_vectors, open("/content/output/metadonnees.tsv", 'w') as file_metadata:
    for word in model.wv.index_to_key:
        file_vectors.write('\t'.join([str(x) for x in model.wv[word]]) + "\n")
        file_metadata.write(word + "\n")

Cette cellule vous permet de télécharger vos résultats pour les mettre ensuite dans le [tensorflow projector](https://projector.tensorflow.org/).

In [ ]:
import shutil
from google.colab import files

zip_base = "/content/output"
shutil.make_archive(zip_base, "zip", export_dir)

zip_path = zip_base + ".zip"
print("Archive créée :", zip_path)

files.download(zip_path)